In [1]:
%load_ext autotime

In [2]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/magellan/')

time: 1.32 ms


In [3]:
# import packages
import pandas as pd
import py_entitymatching as em
import py_stringsimjoin as ssj
import qgrid

time: 1.69 s


In [4]:
# Read tables
A = mg.read_csv_metadata('../DBLP_ACM/DBLP_cleaned.csv', key='id')
B = mg.read_csv_metadata('../DBLP_ACM/ACM_cleaned.csv', key='id')

time: 22.8 ms


In [5]:
# Explore tables
# qgrid.show_grid(A)
# qgrid.show_grid(B)

time: 1.02 ms


In [6]:
# Clean up tables
# Observe that the authors have some ? replace with NaN
A.replace({'authors':{'?':pd.np.NaN}}, inplace=True)
# Normalize attr. values
B.replace({'venue':{
            'The VLDB Journal — The International Journal on Very Large Data Bases':'VLDB J.',
            'Very Large Data Bases': 'VLDB',
            'ACM SIGMOD Record': 'SIGMOD Record'
        }}, inplace=True)

time: 9 ms


In [7]:
# Create attribute equivalence blocker
ab = mg.AttrEquivalenceBlocker()
# Block tables using 'year' attribute: same year then include in the canidate set
C1 = ab.block_tables(A, B, 'year', 'year', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

time: 262 ms


In [8]:
len(C1)

601284

time: 12.4 ms


In [9]:
# Display first two rows from C1
C1.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/sigmod/Mackay99,304586,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",1999
1,1,journals/sigmod/Mackay99,304587,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",1999


time: 38.2 ms


In [10]:
# Debug blocker output
dbg = mg.debug_blocker(C1, A, B, output_size=200)

time: 1.6 s


In [11]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head()

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,1.0,journals/sigmod/Hammer01,601859,Treasurer's Message,Joachim Hammer,SIGMOD Record,Treasurer's message,Joachim Hammer,SIGMOD Record
1,1,1.0,journals/sigmod/Aberer02,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,SIGMOD Record
2,2,1.0,journals/sigmod/Aberer02a,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,SIGMOD Record
3,3,1.0,journals/sigmod/Aberer02b,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,SIGMOD Record
4,4,1.0,journals/sigmod/Aberer03b,601865,Book review column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,SIGMOD Record


time: 15.2 ms


In [12]:
# Create an overlap blocker
ob = mg.OverlapBlocker()
# Apply overlap blocker on 'authors' attribute
C2 = ob.block_tables(A, B, 'authors', 'authors', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


time: 2.07 s


In [13]:
# Check the number of rows in C2
len(C2)

287414

time: 1.83 ms


In [14]:
# Display first two rows from C2
C2.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/vldb/LaurentLSV01,304586,Monotonic complements for independent data warehouses,spyratos lechtenbrger nicolas dominique vossen gottfried jens laurent,2001,The WASA2 object-oriented workflow management system,weske mathias vossen gottfried,1999
1,1,journals/sigmod/McClatcheyV97,304586,Workshop on Workflow Management in Scientific and Engineering Applications - Report,vossen mcclatchey gottfried richard,1997,The WASA2 object-oriented workflow management system,weske mathias vossen gottfried,1999


time: 17.1 ms


In [15]:
# Combine blocker outputs
C = mg.combine_blocker_outputs_via_union([C1, C2])

time: 1.43 s


In [16]:
# Check the number of rows in the consolidated candidate set.
len(C)

857777

time: 2.29 ms


In [17]:
# Debug again
dbg = mg.debug_blocker(C, A, B)

time: 2.92 s


In [18]:
# Display first few rows from the debugger output
dbg.head(10)

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,0.555556,journals/sigmod/Dogac02,945727,Guest Editor's Introduction,Asuman Dogac,SIGMOD Record,Guest editor's introduction,Karl Aberer,SIGMOD Record
1,1,0.555556,journals/sigmod/Dogac98,945727,Guest Editor's Introduction,Asuman Dogac,SIGMOD Record,Guest editor's introduction,Karl Aberer,SIGMOD Record
2,2,0.500000,journals/sigmod/Snodgrass98a,641001,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on influential papers,Kenneth A. Ross,SIGMOD Record
3,3,0.500000,journals/sigmod/Snodgrass98a,602302,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on Influential Papers,Kenneth A. Ross,SIGMOD Record
4,4,0.500000,journals/sigmod/Snodgrass99b,603882,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on influential papers,Kenneth A. Ross,SIGMOD Record
5,5,0.500000,journals/sigmod/Snodgrass99b,507364,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on influential papers,Kenneth A. Ross,SIGMOD Record
6,6,0.500000,journals/sigmod/Snodgrass98a,507364,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on influential papers,Kenneth A. Ross,SIGMOD Record
7,7,0.500000,journals/sigmod/Snodgrass98a,603882,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on influential papers,Kenneth A. Ross,SIGMOD Record
8,8,0.500000,journals/sigmod/Snodgrass99b,604262,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on Influential Papers,Kenneth A. Ross,SIGMOD Record
9,9,0.500000,journals/sigmod/Snodgrass98a,604262,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on Influential Papers,Kenneth A. Ross,SIGMOD Record


time: 19.4 ms


In [19]:
%run demo_blocker.py

time: 2.92 ms


In [24]:
a = 1999
a//100

19

time: 2.38 ms


In [25]:
myab = MyBlocker()
def fn(x):
    
    return x//100
myab.set_blackbox_function(fn)

time: 1.62 ms


In [26]:
C1 = myab.block_tables(A, B, 'year', 'year')

time: 987 ms


In [27]:
C1.head(50)

,_id,ltable_id,rtable_id,ltable_year,rtable_year
0,0,journals/sigmod/Mackay99,304586,1999,1999
1,1,journals/sigmod/Mackay99,304587,1999,1999
2,2,journals/sigmod/Mackay99,304589,1999,1999
3,3,journals/sigmod/Mackay99,304590,1999,1999
4,4,journals/sigmod/Mackay99,304582,1999,1999
5,5,journals/sigmod/Mackay99,304583,1999,1999
6,6,journals/sigmod/Mackay99,304584,1999,1999
7,7,journals/sigmod/Mackay99,304585,1999,1999
8,8,journals/sigmod/Mackay99,306112,1999,1998
9,9,journals/sigmod/Mackay99,306115,1999,1998


time: 70.3 ms
